# Challenge 1

### Imports

In [1]:
import requests
import os
import zipfile
from requests.auth import HTTPBasicAuth
from bs4 import BeautifulSoup

### Log in to hackthissite.org

In [2]:
# Environment variable for the password
username = "FriedrichMaximilian"
password = os.getenv('HTS_PASSWORD')

# URL of the mission page and login page
# URLs
login_url = "https://www.hackthissite.org/user/login"
mission_url = "https://www.hackthissite.org/missions/prog/1/"
submit_url = "https://www.hackthissite.org/missions/prog/1/index.php"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Referer': login_url,
}

# Create a session to persist cookies between requests
session = requests.Session()

# Fetch the login page to get any necessary cookies or hidden fields (e.g., CSRF token)
login_page_response = session.get(login_url, headers=headers)

if login_page_response.status_code == 200:
    print("Login page fetched successfully!")
else:
    print(f"Failed to fetch login page with status code: {login_page_response.status_code}")
    exit()

login_data = {
    'username': username,
    'password': password
}

# Send the POST request to log in
login_response = session.post(login_url, data=login_data, headers=headers)

# Check if login was successful by inspecting the response
if "hello" in login_response.text.lower():
    print("Login successful!")
else:
    print("Login failed.")

# print(login_response.text)



# Print the content of the mission page or process it as needed
# print(mission_page.text)

Login page fetched successfully!
Login successful!


### List of scrambled words

In [3]:
# After logging in, you can access the desired page
mission_page = session.get(mission_url, headers=headers)

# Check if the page was fetched successfully
if mission_page.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(mission_page.text, 'html.parser')

    # Assuming the scrambled words are in a specific tag (like <p> or <span>)
    # Find the section where the scrambled words are located, example:
    scrambled_section = soup.find_all('li')  # Modify this based on actual HTML structure
    
    # Extract and print the scrambled words
    scrambled_words = [word.text for word in scrambled_section]
    
    # Output the extracted scrambled words
    print("Scrambled Words:", scrambled_words[-10:])
else:
    print("Failed to fetch the mission page.")

Scrambled Words: ['rinomna', 'nenszua', 'dienlsy', 'enihktg', 'siucer', 'tkoboi', 'ceikhcn', 'lletcaob', 'dnooln', 'laespe']


### Download wordlist

In [4]:
url = "https://www.hackthissite.org/missions/prog/1/wordlist.zip"
file_name = "wordlist.zip"

# Check if the file already exists
if not os.path.exists(file_name):
    # Send GET request
    response = requests.get(url, stream=True)
    
    # Check if the request was successful
    if response.status_code == 200:
        with open(file_name, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"File downloaded successfully as {file_name}")
    else:
        print(f"Failed to download file. HTTP Status Code: {response.status_code}")
else:
    print(f"File '{file_name}' already exists. Skipping download.")


File 'wordlist.zip' already exists. Skipping download.


### Unzip file

In [5]:
# Extract the ZIP file if not already extracted
extract_folder = "wordlist"
if not os.path.exists(extract_folder):
    with zipfile.ZipFile(file_name, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
        print(f"File extracted to {extract_folder}")
else:
    print(f"Folder '{extract_folder}' already exists. Skipping extraction.")



Folder 'wordlist' already exists. Skipping extraction.


### Read the words

In [6]:
# Open and read the extracted file into a list, line by line
word_list = []
wordlist_path = os.path.join(extract_folder, "wordlist.txt")
if os.path.exists(wordlist_path):
    with open(wordlist_path, "r", encoding="utf-8") as file:
        for line in file:
            word_list.append(line.strip())
            # print(line.strip())  # Process each line as needed
else:
    print(f"Extracted wordlist file '{wordlist_path}' not found.")
# print(word_list)

### Finally, unscramble the words

In [7]:
scrambled_words = scrambled_words[-10:]
word_list = word_list
word_list.sort()
print(scrambled_words, "\n")
# print(word_list)

# restructure the word list according to length of words
d_len = {}

lengths = []
for word in word_list:
    if len(word) not in d_len:
        d_len[len(word)] = [word]
        lengths.append(len(word))
    else:
        d_len[len(word)].append(word)
# print(lengths, "\n")
# print(d_len)
    

ans = []
for scrambled in scrambled_words:
    l = len(scrambled)
    words = d_len[l]
    candidates = []
    i = 0
    candidates = [x for x in words if scrambled[0] in x]
    for ch in scrambled:
        if i == 0:
            i += 1
            continue
        else:
            candidates = [x for x in candidates if ch in x]
    if len(candidates) > 1:
        for can in candidates:
            if sorted(scrambled) == sorted(can):
                candidates = [can]
                break
    # print(candidates)
    # print(scrambled)
    
    ans += candidates
print(len(ans), len(scrambled_words))
ans = ",".join(ans)
print(type(ans), ans)

['rinomna', 'nenszua', 'dienlsy', 'enihktg', 'siucer', 'tkoboi', 'ceikhcn', 'lletcaob', 'dnooln', 'laespe'] 

10 10
<class 'str'> ironman,suzanne,lindsey,theking,cruise,bookit,chicken,tacobell,london,please


### Submit solution

In [8]:
# Data payload containing the solution
payload = {
    "solution": ans, 
    "submitbutton": "submit"  
}

hidden_inputs = soup.find_all("input", type="hidden")
for input_tag in hidden_inputs:
    print(f"🔹 Found hidden field: {input_tag['name']} = {input_tag['value']}")
    payload[input_tag["name"]] = input_tag["value"]
    
# Update headers
headers["Referer"] = mission_url
headers["Content-Type"] = "application/x-www-form-urlencoded"

response = session.post(submit_url, data=payload, headers=headers)

# Debug response
if "incorrect" in response.text.lower():
    print("Submission failed due to incorrect answer")
if "wrong" in response.text.lower():
    print("❌ Wrong answer. Double-check your solution format.")
if "error" in response.text.lower():
    print("❌ Submission failed: error")    
if "congratz" in response.text.lower():
    print("🎉 Submission successful!")
    print("Response URL:", response.url)
    print("Status Code:", response.status_code)
    print(response.text[:2000])  # Show more response data for debugging
elif "sorry" in response.text.lower():
    print("❌ Submission failed: Wrong answer")    
    print("Response URL:", response.url)
    print("Status Code:", response.status_code)
    print(response.text[:2000])  # Show more response data for debugging
else:
    print("❌ Submission failed. Debugging response...")
    print("Response URL:", response.url)
    print("Status Code:", response.status_code)
    print(response.text.lower())  # Show more response data for debugging

print("Session cookies:", session.cookies.get_dict())
print(response)

❌ Submission failed: error
❌ Submission failed. Debugging response...
Response URL: https://www.hackthissite.org/missions/prog/1/index.php
Status Code: 200
<!doctype html public "-//w3c//dtd xhtml 1.0 transitional//en" "http://www.w3.org/tr/xhtml1/dtd/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">

<head>
  <title>hack this site</title>
  <meta name="verify-v1" content="s/yxn7eqrmbof9pl5jljdiwpaxexpjze9jlg/zm4c2y=" />
  <meta http-equiv="content-type" content="text/html; charset=iso-8859-1" />
  <meta name="author" content="hackthissite.org staff" />
  <meta name="keywords" content="challenge, computer, culture, deface, digital, ethics, games, guide, hack, hack forums, hacker, hackers, hacking, hacking challenges, hacking forums, mission, net, programming, radical, revolution, root, rooting, security, site, society, tutorial, tutorials, war, wargame, wargames, web, website" />
  <meta name="title" content="hackthissite" />
  <meta name="de